In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from time import sleep
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
# load in user json
with open('/media/jesse/Number3/json/Jesse.p.tao.json') as f:
    data = json.load(f)
user_df = pd.DataFrame(data['items'])
user_df.head()

,track,played_at,context
0,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T05:19:45.999Z,None
1,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T05:19:26.252Z,None
2,"{'album': {'album_type': 'single', 'artists': ...",2021-02-15T05:18:06.671Z,{'external_urls': {'spotify': 'https://open.sp...
3,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T01:58:55.395Z,None
4,"{'album': {'album_type': 'single', 'artists': ...",2021-02-15T01:45:07.511Z,None


In [3]:
new_features_df = pd.read_csv('../data/new_track_features.csv')
new_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_uri,artist,track_uri,track_name,artist_genres
0,0.804,0.604,7,-6.416,1,0.0390,0.848000,0.000000,0.2460,0.964,105.993,spotify:artist:3CygdxquGHurS7f9LjNLkv,Little Big Town,spotify:track:3VOH59vQXLVTSUxxXcaYiw,"Wine, Beer, Whiskey (Acoustic Version)","['contemporary country', 'country', 'country d..."
1,0.680,0.765,1,-8.981,1,0.7990,0.065000,0.000001,0.0961,0.689,82.688,spotify:artist:6X8WdFjrNhXATMDSs26aCc,Curren$y,spotify:track:55wsrEpFbGhYEAQrfv5jWC,Kush Through the Sunroof,"['alternative hip hop', 'dirty south rap', 'ga..."
2,0.490,0.449,1,-12.361,1,0.2590,0.142000,0.856000,0.1300,0.155,119.847,spotify:artist:5SyGEPymt1G2uto47tVWvZ,MadeinTYO,spotify:track:4cOdRhWoQg8wrtggw6YZLY,HUNNIDDOLLA,"['atl hip hop', 'hip hop', 'melodic rap', 'pop..."
3,0.794,0.833,1,-3.025,1,0.3400,0.300000,0.000000,0.2480,0.584,156.998,spotify:artist:0rpUnzu4JIoRkKkGvdcm7b,Lil Eazzyy,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,Freestyle,"['chicago rap', 'trap']"
4,0.428,0.900,11,-4.195,1,0.0384,0.000016,0.773000,0.2420,0.228,160.057,spotify:artist:56dO9zeHKuU5Gvfc2kxHNw,Chevelle,spotify:track:43WDs22CX8vCalLnpL05T2,Remember When,"['alternative metal', 'alternative rock', 'chr..."


In [4]:
new_track_df = pd.read_csv('../data/tracks_with_popularity.csv')
new_track_df.head()

,artist_uri,artist,track_uri,track_name,track_url,popularity,artist_genres
0,spotify:artist:7bXgB6jMjp9ATFy66eO08Z,Chris Brown,spotify:track:1Jz1yhcPm1Yt6aoed3zact,"Go Crazy (Remix) (feat. Young Thug, Future, Li...",https://open.spotify.com/track/1Jz1yhcPm1Yt6ao...,91,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap']"
1,spotify:artist:64KEffDW9EtZ1y2vBYgq8T,Marshmello,spotify:track:2aLaWfLeSzWrTLdLeVQ6zQ,Lavandia,https://open.spotify.com/track/2aLaWfLeSzWrTLd...,90,"['brostep', 'edm', 'pop', 'pop dance', 'progre..."
2,spotify:artist:6fOMl44jA4Sp5b9PpYCkzz,NF,spotify:track:5UMMPHPp6vRP6ghPpSUOzp,CLOUDS,https://open.spotify.com/track/5UMMPHPp6vRP6gh...,86,"['hip hop', 'pop rap', 'rap']"
3,spotify:artist:6fOMl44jA4Sp5b9PpYCkzz,NF,spotify:track:3K8WGcz1AB6MvSlRypACC8,CLOUDS (Edit),https://open.spotify.com/track/3K8WGcz1AB6MvSl...,86,"['hip hop', 'pop rap', 'rap']"
4,spotify:artist:3tlXnStJ1fFhdScmQeLpuG,Brent Faiyaz,spotify:track:61gjdUMePfAOD5p0gnGvfv,Eden (From “Black History Always / Music For t...,https://open.spotify.com/track/61gjdUMePfAOD5p...,84,"['dmv rap', 'rap']"


In [5]:
new_features_df = pd.merge(new_features_df, new_track_df, how = 'inner')
new_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_uri,artist,track_uri,track_name,artist_genres,track_url,popularity
0,0.804,0.604,7,-6.416,1,0.0390,0.848000,0.000000,0.2460,0.964,105.993,spotify:artist:3CygdxquGHurS7f9LjNLkv,Little Big Town,spotify:track:3VOH59vQXLVTSUxxXcaYiw,"Wine, Beer, Whiskey (Acoustic Version)","['contemporary country', 'country', 'country d...",https://open.spotify.com/track/3VOH59vQXLVTSUx...,70
1,0.680,0.765,1,-8.981,1,0.7990,0.065000,0.000001,0.0961,0.689,82.688,spotify:artist:6X8WdFjrNhXATMDSs26aCc,Curren$y,spotify:track:55wsrEpFbGhYEAQrfv5jWC,Kush Through the Sunroof,"['alternative hip hop', 'dirty south rap', 'ga...",https://open.spotify.com/track/55wsrEpFbGhYEAQ...,69
2,0.490,0.449,1,-12.361,1,0.2590,0.142000,0.856000,0.1300,0.155,119.847,spotify:artist:5SyGEPymt1G2uto47tVWvZ,MadeinTYO,spotify:track:4cOdRhWoQg8wrtggw6YZLY,HUNNIDDOLLA,"['atl hip hop', 'hip hop', 'melodic rap', 'pop...",https://open.spotify.com/track/4cOdRhWoQg8wrtg...,70
3,0.794,0.833,1,-3.025,1,0.3400,0.300000,0.000000,0.2480,0.584,156.998,spotify:artist:0rpUnzu4JIoRkKkGvdcm7b,Lil Eazzyy,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,Freestyle,"['chicago rap', 'trap']",https://open.spotify.com/track/1CnW2I8CBRR8rYx...,70
4,0.428,0.900,11,-4.195,1,0.0384,0.000016,0.773000,0.2420,0.228,160.057,spotify:artist:56dO9zeHKuU5Gvfc2kxHNw,Chevelle,spotify:track:43WDs22CX8vCalLnpL05T2,Remember When,"['alternative metal', 'alternative rock', 'chr...",https://open.spotify.com/track/43WDs22CX8vCalL...,68


In [6]:
new_features_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'artist_uri', 'artist', 'track_uri', 'track_name', 'artist_genres',
       'track_url', 'popularity'],
      dtype='object')

In [7]:
new_features_df = new_features_df.drop(['track_url', 'popularity'], 
                    axis = 1)

In [8]:
new_track_df['track_uri'][0]

'spotify:track:1Jz1yhcPm1Yt6aoed3zact'

In [9]:
user_df.iloc[0]['track']['artists'][0]['id']

'45ft4DyTCEJfQwTBHXpdhM'

In [10]:
track_name = []
artist_uri = []
artist_name = []
track_uri = []
genre = []

for i in range(len(user_df)):
    track_name.append(user_df.iloc[i]['track']['name'])
    track_uri.append('spotify:track:' + user_df.iloc[i]['track']['id'])
    artist_uri.append('spotify:artist:' + user_df.iloc[i]['track']['artists'][0]['id'])
    artist_name.append(user_df.iloc[i]['track']['artists'][0]['name'])
user_df['track_uri'] = track_uri
user_df['track_name'] = track_name
user_df['artist_uri'] = artist_uri
user_df['artist'] = artist_name

# setup spotify with app credentials, use environment variables so GitHub scrapers don't get access to my spotify developer keys
cid = os.getenv('SPOTIPY_CLIENT_ID')
secret = os.getenv('SPOTIPY_CLIENT_SECRET')

#https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b

client_credentials_manager = SpotifyClientCredentials(client_id = cid, 
                                                      client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

for i in tqdm(range(len(user_df))):
     genre.append(sp.artist(user_df.iloc[i]['artist_uri'])['genres'])
     sleep(0.067)
user_df['artist_genres'] = genre

user_df.head()

100%|██████████| 50/50 [00:05<00:00,  8.60it/s]


,track,played_at,context,track_uri,track_name,artist_uri,artist,artist_genres
0,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T05:19:45.999Z,None,spotify:track:22LRgvP6LBtmN5SqGrK3O7,Drown,spotify:artist:45ft4DyTCEJfQwTBHXpdhM,milet,[j-pop]
1,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T05:19:26.252Z,None,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
2,"{'album': {'album_type': 'single', 'artists': ...",2021-02-15T05:18:06.671Z,{'external_urls': {'spotify': 'https://open.sp...,spotify:track:3wWUkfEmfO6qvT1iIrrusD,Too Good to Be True,spotify:artist:5UI7y2lrDYhqcM7sdbTbQ9,xerLK,[background music]
3,"{'album': {'album_type': 'album', 'artists': [...",2021-02-15T01:58:55.395Z,None,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
4,"{'album': {'album_type': 'single', 'artists': ...",2021-02-15T01:45:07.511Z,None,spotify:track:15TaZdhznRBaKzyHLHA9ad,unravel (acoustic version) - Acoustic Version,spotify:artist:3B9O5mYYw89fFXkwKh7jCS,TK from Ling tosite sigure,"[j-poprock, j-rock, otacore]"


In [11]:
user_df.drop(['track', 'played_at', 'context'], axis = 1, inplace = True)

In [12]:
user_df.tail()

,track_uri,track_name,artist_uri,artist,artist_genres
45,spotify:track:2y7f8qkrgIqY5KsfPNcuix,Black Catcher,spotify:artist:0PYPjvZaOa7bsCq26JOX8d,Vickeblanka,"[anime, anime rock, j-pop, j-poprock, otacore]"
46,spotify:track:5CyHyrAW1RPip3ruorIWyo,ワガママMIRROR HEART,spotify:artist:74VIJfMSLnKe5eU3yvv2RT,大橋彩香,"[anime, seiyu]"
47,spotify:track:2MGzJVW4N1SuPGdTlH0fSr,Orange (Shigatsu Wa Kimi No Uso) [Ending],spotify:artist:47bagukDdx1Oqq6aL9JlwE,Berioska,[]
48,spotify:track:3qWtzuw4ma3I3NJUnqHjOs,"Centimeter (Rent a Girlfriend: Kanojo, Okarish...",spotify:artist:1MQ48GaF9nanTisRGu3Mtl,Shayne Orok,[otacore]
49,spotify:track:65tu16zi8H7fGexuPINxJp,"Centimeter (From ""Rent a Girlfriend: Kanojo, O...",spotify:artist:5ZcXM1KFH6hhLoBIdgCnap,Miura Jam,[otacore]


In [13]:
# load in new tracks csv
new_tracks = pd.read_csv('../data/tracks_with_popularity.csv')
new_tracks.tail()

,artist_uri,artist,track_uri,track_name,track_url,popularity,artist_genres
13605,spotify:artist:4ZVipewsvedvy7mASnGs0N,Damek,spotify:track:6NoV4E6EkZry3dmvBJCedC,Пархай,https://open.spotify.com/track/6NoV4E6EkZry3dm...,0,[]
13606,spotify:artist:4fVEFD3EuFjmcuNpIO4K1w,Uğur Çakır,spotify:track:0LVEIh4culsSKnuHczdZLc,Gerçeğim,https://open.spotify.com/track/0LVEIh4culsSKnu...,9,[]
13607,spotify:artist:1t7ZJ7TV6HhEYi0GVUNxtU,Elements 4,spotify:track:0CVrfrTD6zVE0EFmhO72WB,Griffith,https://open.spotify.com/track/0CVrfrTD6zVE0EF...,0,[]
13608,spotify:artist:0h0MuZFryBVHKOjwM0x75m,Reuf Sipovic,spotify:track:0QrH0hLOq6UMYbCns8KgzW,Zajdi Zajdi,https://open.spotify.com/track/0QrH0hLOq6UMYbC...,0,[]
13609,spotify:artist:5M0vStPWGIxiwj4xUWcw43,Sadina,spotify:track:0mNJMpcph6eEdZXLw2P8Y4,Morning,https://open.spotify.com/track/0mNJMpcph6eEdZX...,0,[]


In [14]:
new_tracks = new_tracks[new_tracks['popularity'] <= 70]

In [15]:
# only need to get user features after getting new_track features once
new_features_list = []
user_features_list = []
# for i in tqdm(range(len(new_tracks))):
#     new_features_list.append(sp.audio_features(new_tracks.iloc[i]['track_uri'])[0])
#     sleep(0.02)
for i in tqdm(range(len(user_df))):
    user_features_list.append(sp.audio_features(user_df.iloc[i]['track_uri'])[0])
    sleep(0.02)

100%|██████████| 50/50 [00:03<00:00, 15.19it/s]


In [16]:
def is_empty(any_structure):
    if any_structure:
        return False
    else:
        print('Structure is empty.')
        return True

In [17]:
user_features_list = [i for i in user_features_list if is_empty(i) == False]
new_features_list = [i for i in new_features_list if is_empty(i) == False]

In [18]:
user_features_df = pd.DataFrame(user_features_list)
user_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.437,0.6100,9,-4.212,0,0.0329,0.0102,0.000132,0.3960,0.3510,179.921,audio_features,22LRgvP6LBtmN5SqGrK3O7,spotify:track:22LRgvP6LBtmN5SqGrK3O7,https://api.spotify.com/v1/tracks/22LRgvP6LBtm...,https://api.spotify.com/v1/audio-analysis/22LR...,211787,4
1,0.566,0.9190,2,-1.901,0,0.0639,0.0666,0.000000,0.3400,0.5500,146.990,audio_features,7HAkrW8JUYoVR9dXwb6Uku,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4
2,0.127,0.0838,0,-28.353,1,0.0417,0.9460,0.933000,0.0906,0.0298,146.325,audio_features,3wWUkfEmfO6qvT1iIrrusD,spotify:track:3wWUkfEmfO6qvT1iIrrusD,https://api.spotify.com/v1/tracks/3wWUkfEmfO6q...,https://api.spotify.com/v1/audio-analysis/3wWU...,165205,4
3,0.566,0.9190,2,-1.901,0,0.0639,0.0666,0.000000,0.3400,0.5500,146.990,audio_features,7HAkrW8JUYoVR9dXwb6Uku,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4
4,0.518,0.3900,9,-6.083,1,0.0937,0.8290,0.000000,0.0729,0.3110,84.092,audio_features,15TaZdhznRBaKzyHLHA9ad,spotify:track:15TaZdhznRBaKzyHLHA9ad,https://api.spotify.com/v1/tracks/15TaZdhznRBa...,https://api.spotify.com/v1/audio-analysis/15Ta...,228834,5


In [19]:
user_features_df = pd.merge(user_features_df, user_df, left_on = 'uri', right_on = 'track_uri')
user_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,track_uri,track_name,artist_uri,artist,artist_genres
0,0.437,0.610,9,-4.212,0,0.0329,0.0102,0.000132,0.396,0.351,...,spotify:track:22LRgvP6LBtmN5SqGrK3O7,https://api.spotify.com/v1/tracks/22LRgvP6LBtm...,https://api.spotify.com/v1/audio-analysis/22LR...,211787,4,spotify:track:22LRgvP6LBtmN5SqGrK3O7,Drown,spotify:artist:45ft4DyTCEJfQwTBHXpdhM,milet,[j-pop]
1,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,...,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
2,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,...,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
3,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,...,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
4,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,...,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,https://api.spotify.com/v1/tracks/7HAkrW8JUYoV...,https://api.spotify.com/v1/audio-analysis/7HAk...,242293,4,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"


In [20]:
user_features_df = user_features_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis = 1)
user_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_uri,track_name,artist_uri,artist,artist_genres
0,0.437,0.610,9,-4.212,0,0.0329,0.0102,0.000132,0.396,0.351,179.921,spotify:track:22LRgvP6LBtmN5SqGrK3O7,Drown,spotify:artist:45ft4DyTCEJfQwTBHXpdhM,milet,[j-pop]
1,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,146.990,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
2,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,146.990,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
3,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,146.990,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"
4,0.566,0.919,2,-1.901,0,0.0639,0.0666,0.000000,0.340,0.550,146.990,spotify:track:7HAkrW8JUYoVR9dXwb6Uku,ホタルノヒカリ,spotify:artist:5YneEA2nLtAhkD5t2769lZ,Ikimonogakari,"[anime, j-pop, j-poprock]"


In [21]:
new_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_uri,artist,track_uri,track_name,artist_genres
0,0.804,0.604,7,-6.416,1,0.0390,0.848000,0.000000,0.2460,0.964,105.993,spotify:artist:3CygdxquGHurS7f9LjNLkv,Little Big Town,spotify:track:3VOH59vQXLVTSUxxXcaYiw,"Wine, Beer, Whiskey (Acoustic Version)","['contemporary country', 'country', 'country d..."
1,0.680,0.765,1,-8.981,1,0.7990,0.065000,0.000001,0.0961,0.689,82.688,spotify:artist:6X8WdFjrNhXATMDSs26aCc,Curren$y,spotify:track:55wsrEpFbGhYEAQrfv5jWC,Kush Through the Sunroof,"['alternative hip hop', 'dirty south rap', 'ga..."
2,0.490,0.449,1,-12.361,1,0.2590,0.142000,0.856000,0.1300,0.155,119.847,spotify:artist:5SyGEPymt1G2uto47tVWvZ,MadeinTYO,spotify:track:4cOdRhWoQg8wrtggw6YZLY,HUNNIDDOLLA,"['atl hip hop', 'hip hop', 'melodic rap', 'pop..."
3,0.794,0.833,1,-3.025,1,0.3400,0.300000,0.000000,0.2480,0.584,156.998,spotify:artist:0rpUnzu4JIoRkKkGvdcm7b,Lil Eazzyy,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,Freestyle,"['chicago rap', 'trap']"
4,0.428,0.900,11,-4.195,1,0.0384,0.000016,0.773000,0.2420,0.228,160.057,spotify:artist:56dO9zeHKuU5Gvfc2kxHNw,Chevelle,spotify:track:43WDs22CX8vCalLnpL05T2,Remember When,"['alternative metal', 'alternative rock', 'chr..."


In [22]:
combined_features_df = pd.concat([new_features_df, user_features_df])
combined_features_df.reset_index(drop = True, inplace = True)
combined_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_uri,artist,track_uri,track_name,artist_genres
0,0.804,0.604,7,-6.416,1,0.0390,0.848000,0.000000,0.2460,0.964,105.993,spotify:artist:3CygdxquGHurS7f9LjNLkv,Little Big Town,spotify:track:3VOH59vQXLVTSUxxXcaYiw,"Wine, Beer, Whiskey (Acoustic Version)","['contemporary country', 'country', 'country d..."
1,0.680,0.765,1,-8.981,1,0.7990,0.065000,0.000001,0.0961,0.689,82.688,spotify:artist:6X8WdFjrNhXATMDSs26aCc,Curren$y,spotify:track:55wsrEpFbGhYEAQrfv5jWC,Kush Through the Sunroof,"['alternative hip hop', 'dirty south rap', 'ga..."
2,0.490,0.449,1,-12.361,1,0.2590,0.142000,0.856000,0.1300,0.155,119.847,spotify:artist:5SyGEPymt1G2uto47tVWvZ,MadeinTYO,spotify:track:4cOdRhWoQg8wrtggw6YZLY,HUNNIDDOLLA,"['atl hip hop', 'hip hop', 'melodic rap', 'pop..."
3,0.794,0.833,1,-3.025,1,0.3400,0.300000,0.000000,0.2480,0.584,156.998,spotify:artist:0rpUnzu4JIoRkKkGvdcm7b,Lil Eazzyy,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,Freestyle,"['chicago rap', 'trap']"
4,0.428,0.900,11,-4.195,1,0.0384,0.000016,0.773000,0.2420,0.228,160.057,spotify:artist:56dO9zeHKuU5Gvfc2kxHNw,Chevelle,spotify:track:43WDs22CX8vCalLnpL05T2,Remember When,"['alternative metal', 'alternative rock', 'chr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13512,0.592,0.894,11,-4.710,0,0.0420,0.003580,0.000005,0.0721,0.642,100.026,spotify:artist:0PYPjvZaOa7bsCq26JOX8d,Vickeblanka,spotify:track:2y7f8qkrgIqY5KsfPNcuix,Black Catcher,"[anime, anime rock, j-pop, j-poprock, otacore]"
13513,0.395,0.953,8,-3.845,1,0.0914,0.045300,0.000006,0.1250,0.631,90.072,spotify:artist:74VIJfMSLnKe5eU3yvv2RT,大橋彩香,spotify:track:5CyHyrAW1RPip3ruorIWyo,ワガママMIRROR HEART,"[anime, seiyu]"
13514,0.393,0.413,6,-7.238,1,0.0304,0.640000,0.000000,0.0761,0.247,85.820,spotify:artist:47bagukDdx1Oqq6aL9JlwE,Berioska,spotify:track:2MGzJVW4N1SuPGdTlH0fSr,Orange (Shigatsu Wa Kimi No Uso) [Ending],[]
13515,0.593,0.844,2,-6.716,1,0.0465,0.006710,0.000477,0.0739,0.794,139.936,spotify:artist:1MQ48GaF9nanTisRGu3Mtl,Shayne Orok,spotify:track:3qWtzuw4ma3I3NJUnqHjOs,"Centimeter (Rent a Girlfriend: Kanojo, Okarish...",[otacore]


In [24]:
compare_df = combined_features_df[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'mode']]
mms = MinMaxScaler()
compare_df_sc = mms.fit_transform(compare_df)
compare_df_sc = pd.DataFrame(compare_df_sc, columns = compare_df.columns)
compare_df_sc.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode
0,0.804176,0.603906,0.810512,0.016931,0.851406,0.000000,0.238765,0.970796,0.366448,1.0
1,0.670021,0.764944,0.756717,0.821164,0.065261,0.000001,0.086008,0.693857,0.239377,1.0
2,0.464460,0.448869,0.685829,0.249735,0.142570,0.856000,0.120554,0.156093,0.441988,1.0
3,0.793357,0.832960,0.881630,0.335450,0.301205,0.000000,0.240803,0.588117,0.644555,1.0
4,0.397382,0.899976,0.857092,0.016296,0.000016,0.773000,0.234689,0.229607,0.661234,1.0


In [25]:
def compute_distance(a, b):
    normalize_a = tf.nn.l2_normalize(a, 1)
    normalize_b = tf.nn.l2_normalize(b, 1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b = True)
    return distance

In [26]:
%%time
distances = compute_distance(compare_df_sc, compare_df_sc)

CPU times: user 961 ms, sys: 744 ms, total: 1.7 s
Wall time: 1.7 s


In [22]:
%%time
cosine_similarity(compare_df_sc)

CPU times: user 5.87 s, sys: 18.8 s, total: 24.7 s
Wall time: 1.13 s


array([[1.        , 0.75983239, 0.8230966 , ..., 0.91092956, 0.9640331 ,
        0.96148194],
       [0.75983239, 1.        , 0.88242845, ..., 0.75263493, 0.72995186,
        0.73338118],
       [0.8230966 , 0.88242845, 1.        , ..., 0.65899406, 0.79243338,
        0.79999897],
       ...,
       [0.91092956, 0.75263493, 0.65899406, ..., 1.        , 0.85597416,
        0.84987196],
       [0.9640331 , 0.72995186, 0.79243338, ..., 0.85597416, 1.        ,
        0.99293134],
       [0.96148194, 0.73338118, 0.79999897, ..., 0.84987196, 0.99293134,
        1.        ]])

In [27]:
distances_df = pd.DataFrame(distances.numpy(), index = combined_features_df['track_uri'], columns = combined_features_df['track_uri'])

In [28]:
distances_df.tail()

track_uri,spotify:track:3VOH59vQXLVTSUxxXcaYiw,spotify:track:55wsrEpFbGhYEAQrfv5jWC,spotify:track:4cOdRhWoQg8wrtggw6YZLY,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,spotify:track:43WDs22CX8vCalLnpL05T2,spotify:track:6eBNUcu4T5tWhE0rGDwtTP,spotify:track:6uCyWZTj2bV94a5vkjZkik,spotify:track:1Gd8otKnfP6qwdFphyKEZo,spotify:track:4UikQCMenLeOaYzTSgd8s4,spotify:track:1CIxzzS0nVJNfk5wuN2MjM,...,spotify:track:4YXlnwnK3yt5xLfF5Q8wKG,spotify:track:5hbg2YisSRgoGG85pl0g1F,spotify:track:3mScGCzxiXA9OaHdBeuk7O,spotify:track:21z1GxeHjfD3qkaF5CgCTO,spotify:track:2XpV9sHBexcNrz0Gyf3l18,spotify:track:2y7f8qkrgIqY5KsfPNcuix,spotify:track:5CyHyrAW1RPip3ruorIWyo,spotify:track:2MGzJVW4N1SuPGdTlH0fSr,spotify:track:3qWtzuw4ma3I3NJUnqHjOs,spotify:track:65tu16zi8H7fGexuPINxJp
track_uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:2y7f8qkrgIqY5KsfPNcuix,0.251665,0.251527,0.428785,0.181128,0.291760,0.192707,0.031612,0.367441,0.397321,0.323711,...,0.203570,0.145243,0.211701,0.205028,0.018451,2.220446e-16,0.176280,0.381744,0.160037,0.159410
spotify:track:5CyHyrAW1RPip3ruorIWyo,0.130825,0.096951,0.211775,0.059698,0.127899,0.053168,0.215090,0.234987,0.227306,0.065290,...,0.049433,0.018050,0.054433,0.019598,0.187224,1.762801e-01,0.000000,0.131841,0.023109,0.026795
spotify:track:2MGzJVW4N1SuPGdTlH0fSr,0.079747,0.205846,0.202412,0.106989,0.210008,0.125538,0.369898,0.153414,0.245496,0.089594,...,0.157796,0.157805,0.157734,0.134252,0.374080,3.817442e-01,0.131841,0.000000,0.152619,0.158827
spotify:track:3qWtzuw4ma3I3NJUnqHjOs,0.107070,0.101974,0.210334,0.039541,0.133416,0.017100,0.169670,0.192561,0.205323,0.086741,...,0.046867,0.003570,0.057409,0.051670,0.152430,1.600374e-01,0.023109,0.152619,0.000000,0.007124
spotify:track:65tu16zi8H7fGexuPINxJp,0.103738,0.107128,0.211653,0.035436,0.128848,0.017728,0.168144,0.184514,0.206191,0.088579,...,0.053639,0.002987,0.051921,0.040728,0.156218,1.594099e-01,0.026795,0.158827,0.007124,0.000000


In [29]:
distances_df.shape

(13517, 13517)

In [30]:
distances_df.tail(len(user_features_list))

track_uri,spotify:track:3VOH59vQXLVTSUxxXcaYiw,spotify:track:55wsrEpFbGhYEAQrfv5jWC,spotify:track:4cOdRhWoQg8wrtggw6YZLY,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,spotify:track:43WDs22CX8vCalLnpL05T2,spotify:track:6eBNUcu4T5tWhE0rGDwtTP,spotify:track:6uCyWZTj2bV94a5vkjZkik,spotify:track:1Gd8otKnfP6qwdFphyKEZo,spotify:track:4UikQCMenLeOaYzTSgd8s4,spotify:track:1CIxzzS0nVJNfk5wuN2MjM,...,spotify:track:4YXlnwnK3yt5xLfF5Q8wKG,spotify:track:5hbg2YisSRgoGG85pl0g1F,spotify:track:3mScGCzxiXA9OaHdBeuk7O,spotify:track:21z1GxeHjfD3qkaF5CgCTO,spotify:track:2XpV9sHBexcNrz0Gyf3l18,spotify:track:2y7f8qkrgIqY5KsfPNcuix,spotify:track:5CyHyrAW1RPip3ruorIWyo,spotify:track:2MGzJVW4N1SuPGdTlH0fSr,spotify:track:3qWtzuw4ma3I3NJUnqHjOs,spotify:track:65tu16zi8H7fGexuPINxJp
track_uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:7HAkrW8JUYoVR9dXwb6Uku,0.255631,0.274640,0.402029,0.160822,0.254926,0.190353,0.030891,0.346125,0.407072,0.323825,...,1.753992e-01,0.149382,1.675686e-01,1.945532e-01,0.030081,2.854900e-02,0.194003,0.366406,0.173003,0.153968
spotify:track:7HAkrW8JUYoVR9dXwb6Uku,0.255631,0.274640,0.402029,0.160822,0.254926,0.190353,0.030891,0.346125,0.407072,0.323825,...,1.753992e-01,0.149382,1.675686e-01,1.945532e-01,0.030081,2.854900e-02,0.194003,0.366406,0.173003,0.153968
spotify:track:7HAkrW8JUYoVR9dXwb6Uku,0.255631,0.274640,0.402029,0.160822,0.254926,0.190353,0.030891,0.346125,0.407072,0.323825,...,1.753992e-01,0.149382,1.675686e-01,1.945532e-01,0.030081,2.854900e-02,0.194003,0.366406,0.173003,0.153968
spotify:track:3wWUkfEmfO6qvT1iIrrusD,0.358432,0.540562,0.184320,0.415354,0.282757,0.455865,0.710035,0.354392,0.281460,0.433846,...,4.505392e-01,0.503062,4.506405e-01,4.556844e-01,0.706854,7.714441e-01,0.497404,0.256251,0.490823,0.496265
spotify:track:15TaZdhznRBaKzyHLHA9ad,0.061707,0.209825,0.226292,0.115555,0.254965,0.146653,0.362245,0.115951,0.261133,0.130874,...,2.016148e-01,0.189305,1.989620e-01,1.784071e-01,0.377061,3.868299e-01,0.169761,0.008392,0.181964,0.187917
spotify:track:0npl1hbxBVDkdqZezo2wyn,0.224549,0.256022,0.414077,0.168712,0.308054,0.167999,0.018837,0.320383,0.361442,0.317548,...,2.245794e-01,0.156441,2.306646e-01,2.339071e-01,0.034110,1.737902e-02,0.209424,0.366745,0.165517,0.163637
spotify:track:3D9yweVjHsSyfINp2Kbyx8,0.302333,0.307067,0.417880,0.192318,0.263865,0.207098,0.046260,0.388105,0.446034,0.363332,...,1.608040e-01,0.159850,1.761854e-01,2.420566e-01,0.022321,5.433183e-02,0.214913,0.401039,0.172905,0.176408
spotify:track:0PyCEbEnaoodIqPCLsFxG2,0.144551,0.104566,0.194594,0.037022,0.103731,0.037390,0.166762,0.220322,0.232022,0.083847,...,1.124734e-02,0.014438,1.859489e-02,3.801787e-02,0.152835,1.665339e-01,0.021884,0.147368,0.015405,0.020856
spotify:track:28DlaPydCnrs8NxYOnUPZ8,0.131021,0.092515,0.197781,0.047888,0.118934,0.037484,0.193195,0.234951,0.199416,0.050759,...,4.410614e-02,0.016661,4.885563e-02,1.717129e-02,0.183085,1.636582e-01,0.004452,0.132233,0.018218,0.022414


In [31]:
distances_df.loc['score'] = distances_df.tail(len(user_features_list)).sum()
distances_df.loc['score'][:-len(user_features_list)]

track_uri
spotify:track:3VOH59vQXLVTSUxxXcaYiw    10.162539
spotify:track:55wsrEpFbGhYEAQrfv5jWC    11.055973
spotify:track:4cOdRhWoQg8wrtggw6YZLY    14.677471
spotify:track:1CnW2I8CBRR8rYxTcPP7Bn     6.585437
spotify:track:43WDs22CX8vCalLnpL05T2    10.211608
                                          ...    
spotify:track:0CVrfrTD6zVE0EFmhO72WB     9.593059
spotify:track:0QrH0hLOq6UMYbCns8KgzW    12.644945
spotify:track:0mNJMpcph6eEdZXLw2P8Y4    10.380344
spotify:track:22LRgvP6LBtmN5SqGrK3O7     9.075097
spotify:track:7HAkrW8JUYoVR9dXwb6Uku     7.360402
Name: score, Length: 13467, dtype: float64

In [32]:
distances_df.loc['artist'] = combined_features_df['artist'].values
distances_df.loc['artist_uri'] = combined_features_df['artist_uri'].values
distances_df.loc['track_name'] = combined_features_df['track_name'].values
distances_df.loc['artist_genres'] = combined_features_df['artist_genres'].values

In [37]:
len(combined_features_df['artist'].values)

13517

In [59]:
all_score_df = distances_df.loc[['score', 'artist', 'artist_uri', 'track_name', 'artist_genres']].iloc[:, :-len(user_features_df)]
all_score_df.head()

track_uri,spotify:track:3VOH59vQXLVTSUxxXcaYiw,spotify:track:55wsrEpFbGhYEAQrfv5jWC,spotify:track:4cOdRhWoQg8wrtggw6YZLY,spotify:track:1CnW2I8CBRR8rYxTcPP7Bn,spotify:track:43WDs22CX8vCalLnpL05T2,spotify:track:6eBNUcu4T5tWhE0rGDwtTP,spotify:track:6uCyWZTj2bV94a5vkjZkik,spotify:track:1Gd8otKnfP6qwdFphyKEZo,spotify:track:4UikQCMenLeOaYzTSgd8s4,spotify:track:1CIxzzS0nVJNfk5wuN2MjM,...,spotify:track:1vlCUNV5bv5AJwGDmpA2tl,spotify:track:5imQwKYoOQIsME7BkeZWE6,spotify:track:6PhA8SMyoar2Xun7UFgRVQ,spotify:track:4bingaLKYkIiAJp9ZkvSzN,spotify:track:2D4I63FyPezrQ6vG7TJ8qY,spotify:track:6NoV4E6EkZry3dmvBJCedC,spotify:track:0LVEIh4culsSKnuHczdZLc,spotify:track:0CVrfrTD6zVE0EFmhO72WB,spotify:track:0QrH0hLOq6UMYbCns8KgzW,spotify:track:0mNJMpcph6eEdZXLw2P8Y4
track_uri,,,,,,,,,,,,,,,,,,,,,
score,10.162539,11.055973,14.677471,6.585437,10.211608,7.241595,8.154788,14.125123,15.320383,10.980793,...,7.57594,10.024291,12.375535,11.78693,9.151689,9.31217,10.325886,9.593059,12.644945,10.380344
artist,Little Big Town,Curren$y,MadeinTYO,Lil Eazzyy,Chevelle,TobyMac,BIG30,BIG30,Christopher Jackson,Lund,...,Fulcanelli,Marlix,Wugori,Hartung,Karol Freitas,Damek,Uğur Çakır,Elements 4,Reuf Sipovic,Sadina
artist_uri,spotify:artist:3CygdxquGHurS7f9LjNLkv,spotify:artist:6X8WdFjrNhXATMDSs26aCc,spotify:artist:5SyGEPymt1G2uto47tVWvZ,spotify:artist:0rpUnzu4JIoRkKkGvdcm7b,spotify:artist:56dO9zeHKuU5Gvfc2kxHNw,spotify:artist:5VX8hxrcfJWwaTLiqGUHG3,spotify:artist:4nZmMrwH6LxHnCzQv4IFUE,spotify:artist:4nZmMrwH6LxHnCzQv4IFUE,spotify:artist:6sLwRSXSUF5JTUnQaFenyj,spotify:artist:5uLkZ7FWMTBUg1corQqdkR,...,spotify:artist:6BnxPuUTX2010RWNZgIPRh,spotify:artist:3noIEyzpHrth3e22dkm5CX,spotify:artist:5vg8lNOrJ5RRlo6ROXVyox,spotify:artist:6Jvr25FfMR1YnsMQ5lcTpX,spotify:artist:2RcRgmU5UMASF2FSlz0eag,spotify:artist:4ZVipewsvedvy7mASnGs0N,spotify:artist:4fVEFD3EuFjmcuNpIO4K1w,spotify:artist:1t7ZJ7TV6HhEYi0GVUNxtU,spotify:artist:0h0MuZFryBVHKOjwM0x75m,spotify:artist:5M0vStPWGIxiwj4xUWcw43
track_name,"Wine, Beer, Whiskey (Acoustic Version)",Kush Through the Sunroof,HUNNIDDOLLA,Freestyle,Remember When,Help Is On The Way (Maybe Midnight),Blrrrd Pt 2,Rock Out,River,Reckless,...,CYF,Dzīves Seriāls,Danny Phantom,Let Her Go,Pagando Patos,Пархай,Gerçeğim,Griffith,Zajdi Zajdi,Morning
artist_genres,"['contemporary country', 'country', 'country d...","['alternative hip hop', 'dirty south rap', 'ga...","['atl hip hop', 'hip hop', 'melodic rap', 'pop...","['chicago rap', 'trap']","['alternative metal', 'alternative rock', 'chr...","['ccm', 'christian alternative rock', 'christi...","['memphis hip hop', 'trap']","['memphis hip hop', 'trap']","['hollywood', 'show tunes']","['emo rap', 'lo-fi chill', 'pop rap', 'sad rap']",...,[],[],[],[],['musica feirense'],[],[],[],[],[]


In [47]:
column_store = []

In [54]:
for i in column_store:
    print(i[0])

10.162538801561517
11.05597283788595
14.677471203776067
6.585437440264707
10.211608033556795
7.241594830659357
8.154788091957904
14.125123175386703
15.320383444461404
10.980793477089291
6.3463314891487395
7.101743009981173
6.607769724414421
15.207245898609695
6.504681427589255
7.591488365562894
8.080122716236461
15.818237140375343
20.329927814864647
24.281264900774985
19.867317148352065
18.481218896698763
6.3988474533538895
9.49395472030154
9.928853535081302
8.750913932002403
8.420345286250807
10.373951811958035
10.288351932255617
10.046063953832704
9.554666650019662
10.679482220026268
8.188744791331535
6.4865378492677035
8.629814469266789
12.242141881369678
14.873967759271569
9.049953214202624
7.51728492142009
26.656624249615156
13.19292448951227
11.06685818878182
9.381836219081777
14.751409025585101
9.868730100813364
16.91534430765517
12.068947164089723
11.559021551800788
18.334397448732805
22.04438771024689
14.946763653244732
10.011375005791894
6.837606757971468
9.419393344511494
10

In [48]:
for _, values in all_score_df.iteritems():
    column_store.append(values)

In [44]:
all_score_df['spotify:track:3VOH59vQXLVTSUxxXcaYiw']

track_uri
score                                                    10.162539
artist                                             Little Big Town
artist_uri                   spotify:artist:3CygdxquGHurS7f9LjNLkv
track_name                  Wine, Beer, Whiskey (Acoustic Version)
artist_genres    ['contemporary country', 'country', 'country d...
Name: spotify:track:3VOH59vQXLVTSUxxXcaYiw, dtype: object

In [77]:
all_score_df['spotify:track:3VOH59vQXLVTSUxxXcaYiw']

track_uri
score                                                    10.162539
artist                                             Little Big Town
artist_uri                   spotify:artist:3CygdxquGHurS7f9LjNLkv
track_name                  Wine, Beer, Whiskey (Acoustic Version)
artist_genres    ['contemporary country', 'country', 'country d...
Name: spotify:track:3VOH59vQXLVTSUxxXcaYiw, dtype: object

In [ ]:
for i in distances_df.loc['score'][:-len(user_features_list)].sort_values()[0:5].index

In [ ]:
sp.track(distances_df.loc['score'][:-len(user_features_list)].sort_values()[0].index

In [ ]:
distances_df.loc['score'][:-len(user_features_list)].sort_values()[0:5].index

In [30]:
distances_df.loc['score'][:-len(user_features_list)].sort_values()[0:5]

uri
spotify:track:7zY35cpncy3UYs1ZBfASGW    5.656621
spotify:track:0DT1nb0hytfdzS1J9piX1W    5.723181
spotify:track:4IvzoLnAzFneKNpikShYMu    5.769391
spotify:track:0J71CQRti8BcKzdwlssDDG    5.776900
spotify:track:43HTmB9DuOU7DCzhK9bIqz    5.786268
Name: score, dtype: float64